In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
# client.load_world('Town10HD_Opt')

In [4]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [5]:
vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [6]:
# RGB camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.1))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [7]:
time.sleep(0.2)
spectator.set_transform(camera.get_transform())
# camera.destroy()

In [8]:
# camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame)) # Save camera output

# camera.stop() # Stop camera operation

In [9]:
def camera_callback(image, data_dict):
    data_dict['image'] = np.copy(image.raw_data).reshape((image.height, image.width, 4)).astype(np.uint8)

In [10]:
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}

camera.listen(lambda image: camera_callback(image, camera_data))

In [11]:
traffic_manager = client.get_trafficmanager()
vehicle.set_autopilot(True, traffic_manager.get_port())

In [ ]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

while True:
    cv2.imshow('RGB Camera', camera_data['image'])

    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()